In [52]:
from pathlib import Path
import os
import json
import numpy as np
import pandas as pd

In [53]:
cwd = Path().resolve()
result_path = cwd.joinpath('result')

majors = os.listdir(result_path)

compulstory = set()
elective = set()

for major in majors:
    major_path = result_path.joinpath(major)
    for wajib in os.listdir(major_path.joinpath('wajib')):
        compulstory.add(wajib)

    for pilihan in os.listdir(major_path.joinpath('pilihan')):
        elective.add(pilihan)

print(f"Majors count {len(majors)}")
print(f"Compulsory count {compulstory.__len__()}")
print(f"Elective count {len(elective)}")

print(f"Total {len(majors) + len(compulstory) + len(elective)}")

Majors count 50
Compulsory count 1584
Elective count 1120
Total 2754


In [54]:
content = None

with open("major_subjects_keyword.json") as r:
    content = json.load(r)

keywords_length = []
extracted_keywords = []
subjects = []

for major in content:
    major_keyword_counter = 0
    for subject in major["subjects"]:
        for keyword in subject['keywords']:
            keywords_length.append(len(keyword))
        keywords = len(subject['keywords'])
        major_keyword_counter+=keywords
        extracted_keywords.append(keywords)

    subjects.append(major_keyword_counter)

print("SUBJECT KEYWORDS")
keywords_pd = pd.DataFrame(np.array(extracted_keywords))
keywords_pd.describe()


# print(f"Keywords count {len(extracted_keywords)}")
# print(f"Subjects (not unique) count {len(subjects)}")
# print(f"Average keywords per subject")


SUBJECT KEYWORDS


,0
count,2810.000000
mean,9.204626
std,6.519608
min,0.000000
25%,5.000000
50%,8.000000
75%,12.000000
max,61.000000


In [55]:
keywords_pd.sum()

0    25865
dtype: int64

In [56]:
print("Major keywords")
majors_pd = pd.DataFrame(np.array(subjects))
majors_pd.describe()


Major keywords


,0
count,50.000000
mean,517.300000
std,154.671508
min,226.000000
25%,427.000000
50%,506.500000
75%,600.750000
max,986.000000


In [57]:
majors_pd.sum()

0    25865
dtype: int64

In [58]:
print(keywords)

keywords_length_pd = pd.DataFrame(np.array(keywords_length))
keywords_length_pd.describe()

13


,0
count,25865.000000
mean,29.879992
std,24.620447
min,5.000000
25%,16.000000
50%,22.000000
75%,34.000000
max,639.000000


In [59]:
weight = []
weight_normalized = []

with open('weight.txt') as r:
    content = r.read().split('\n')

    for line in content:
        weight.append(float(line.split(' ')[-1]))

with open('weight_normalized.txt') as r:
    content = r.read().split('\n')

    for line in content:
        weight_normalized.append(float(line.split(' ')[-1]))

In [60]:
weight_pd = pd.DataFrame(weight)
weight_pd.describe()

,0
count,1225.000000
mean,0.939700
std,0.029977
min,0.801619
25%,0.922746
50%,0.943052
75%,0.961652
max,0.995614


In [61]:
weight_normalized_pd = pd.DataFrame(weight_normalized)
weight_normalized_pd.describe()

,0
count,1225.000000
mean,71.177890
std,15.452411
min,0.000000
25%,62.438108
50%,72.905751
75%,82.493678
max,100.000000


In [62]:
reader = open('faculty_major.json')
data = json.load(reader)
reader.close()

majors = {}

for each in data:
    majs = each["majors"]

    for major in majs:
        majors[str(major["code"])] = major["acronym"]

In [63]:
import networkx as nx
from typing import List, Tuple

reader = open('weight_normalized.txt')
raw = reader.read().split('\n')
reader.close()

data: List[Tuple[str,str,float]] = []

nodes = set()

for each in raw:
    splitted = each.split(' ')
    node_a = majors[splitted[0].split("-")[-1]]
    node_b = majors[splitted[1].split('-')[-1]]
    nodes.add(node_a)
    nodes.add(node_b)
    number = float(splitted[2])# rescale to 1-10
    data.append((node_a, node_b, number))

nodes = list(nodes)

In [73]:
G = nx.Graph()

selected_nodes = list(nodes)[:5]

for node in selected_nodes:
    G.add_node(node)

for edge in data:
    node_a, node_b, weight = edge
    if node_a in selected_nodes and node_b in selected_nodes:
        G.add_edge(node_a, node_b, weight=weight)

mat = nx.adjacency_matrix(G)
np.set_printoptions(precision=2, threshold=np.inf)
mat.todense()
# print(mat.shape)

/tmp/ipykernel_801/4043584113.py:13: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  mat = nx.adjacency_matrix(G)


matrix([[ 0.  , 81.89, 65.54, 95.59, 61.95],
        [81.89,  0.  , 89.24, 71.46, 83.98],
        [65.54, 89.24,  0.  , 53.63, 70.98],
        [95.59, 71.46, 53.63,  0.  , 61.58],
        [61.95, 83.98, 70.98, 61.58,  0.  ]])

In [ ]:
print(list(G.nodes))